In [1]:
from pathlib import Path
import math
import pandas as pd
from neptoon.quality_control import (
    QualityAssessmentFlagBuilder,
    FlagRangeCheck,
    FlagSpikeDetectionUniLOF,
)
from neptoon.hub import CRNSDataHub
from neptoon.config import SiteInformation
from neptoon.corrections import (
    CorrectionType,
    CorrectionTheory,
    Correction,
)
from neptoon.data_audit import (
    DataAuditLog,
)
from neptoon.io.read import (
    CollectAndParseRawData
)

In [ ]:
DataAuditLog.create()

In [ ]:
abspath = "" 
data_creator = CollectAndParseRawData(
    path_to_yaml= abspath + "configuration_files/A101_station.yaml"
)
crns_df = data_creator.create_data_frame()

In [ ]:
site_information = SiteInformation(
    site_name="x_some_site",
    latitude=51.37,
    longitude=12.55,
    elevation=140,
    reference_incoming_neutron_value=150,
    dry_soil_bulk_density=1.4,
    lattice_water=0.01,
    soil_organic_carbon=0,
    n0=700,
    cutoff_rigidity=2.94,
    site_biomass = 1
)

# site_information.add_custom_value("n0", 1500)
site_information.add_custom_value("biomass", 1)


In [ ]:
data_hub = CRNSDataHub(
    crns_data_frame=crns_df, site_information=site_information
)
data_hub.validate_dataframe(schema="initial_check")

The `attach_nmdb_data()` method has defaults. Running the below code, without selecting values, will work too. 


```python
data_hub.attach_nmdb_data()
```



In [ ]:
data_hub.attach_nmdb_data(
    station = "JUNG",
    new_column_name = "incoming_neutron_intensity",
    resolution = "60",
    nmdb_table = "revori"
    )
data_hub.crns_data_frame

In [ ]:
data_hub.prepare_static_values()
data_hub.crns_data_frame

In [ ]:
qa_flags = QualityAssessmentFlagBuilder()
qa_flags.add_check(
    FlagRangeCheck("air_relative_humidity", min_val=0, max_val=100),
    # FlagRangeCheck("precipitation", min_val=0, max_val=20),
    FlagSpikeDetectionUniLOF("epithermal_neutrons_cph"),
    # ...
)

data_hub.add_quality_flags(custom_flags=qa_flags)
data_hub.apply_quality_flags()
data_hub.flags_data_frame


In [ ]:
data_hub.select_correction(
    correction_type=CorrectionType.INCOMING_INTENSITY,
    correction_theory=CorrectionTheory.ZREDA_2012,
)
data_hub.select_correction(
    correction_type=CorrectionType.HUMIDITY,
    correction_theory=CorrectionTheory.ROSOLEM_2013
)

data_hub.select_correction(
    correction_type=CorrectionType.PRESSURE,
)

# data_hub.select_correction(
#     correction_type=CorrectionType.ABOVE_GROUND_BIOMASS
# )

data_hub.correct_neutrons()
data_hub.crns_data_frame

In [ ]:
class NewIdeaForBiomass(Correction):
    """
    My new idea to correct for biomass with humidity
    """

    def __init__(self, 
                site_information,
                correction_type = CorrectionType.CUSTOM,
                 
                correction_factor_column_name: str = "new_biomass_correction",
                ):
        super().__init__(correction_type=correction_type, 
                         correction_factor_column_name=correction_factor_column_name)
        self.site_information = site_information
        self.humidity_column_name = "air_relative_humidity"

    @staticmethod
    def new_func(biomass, humidity):
        if biomass == 0:
            return 1
        return 1-((biomass / humidity) / 1000)

    def apply(self, data_frame: pd.DataFrame):

        data_frame[self.correction_factor_column_name] = data_frame.apply(
            lambda row: self.new_func(
                row[self.humidity_column_name],
                self.site_information.site_biomass, # Here we use a value from site_information
            ),
            axis=1,
        )
        return data_frame

In [ ]:
data_hub.correction_factory.register_custom_correction(
    correction_type=CorrectionType.CUSTOM,
    theory="my_new_idea",
    correction_class=NewIdeaForBiomass,
)

data_hub.select_correction(
    correction_type=CorrectionType.CUSTOM,
    correction_theory="my_new_idea",
)

data_hub.prepare_static_values()
data_hub.correct_neutrons()
data_hub.crns_data_frame

In [ ]:
data_hub.produce_soil_moisture_estimates()
data_hub.crns_data_frame

In [ ]:
df = data_hub.crns_data_frame

data_hub.save_data(append_yaml_hash_to_folder_name=True)